In [ ]:
import pymongo
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.model_selection import LeaveOneOut, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE

def corregir_dtypes_dataframe(df):
    df.cantidad = pd.to_numeric(df.cantidad)
    return df

def crear_dataframe_desde_mongodb():
    cadena_conexion = ('mongodb://{nom_usuario}:{password}@{host}:{port}')
    cadena_conexion = cadena_conexion.format(
        nom_usuario = 'PY01_c02',
        password = 'P4rd83XkXrTz',
        host = '5.189.129.12',
        port = 27017
    )
    client = pymongo.MongoClient(cadena_conexion)
    db = client.PY01
    header = [*db.PY01.find_one().keys()]
    lista_dataset = list()
    for doc in db.PY01.find():
        lista_dataset.append([*doc.values()])
 
    df = pd.DataFrame(data = lista_dataset, columns = header)
    return df

def limpiar_dataset(df):
    dias = {'LUNES': 1, 'MONDAY': 1, 'MARTES': 2, 'TUESDAY': 2, 'MIERCOLES': 3, 'WEDNESDAY': 3, 'JUEVES': 4, 'THURSDAY': 4, 'VIERNES': 5, 'FRIDAY': 5, 'SABADO': 6, 'SATURDAY': 6, 'DOMINGO': 7, 'SUNDAY': 7}
    meses = {'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4, 'Mayo': 5, 'Junio': 6, 'Julio': 7, 'Agosto': 8, 'Setiembre': 9, 'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12}
    for dia, cod in dias.items():
        df.loc[df.dia == dia, 'dia'] = cod
    df.dia = pd.to_numeric(df.dia)
    
    for mes, cod in meses.items():
        df.loc[df.mes == mes, 'mes'] = cod
    df.mes = pd.to_numeric(df.mes)
    
    return df

def generar_variables(df):
    nuevos_registros = list()
    for row in df.itertuples():
        cantidad = row.cantidad
        if cantidad > 1:
            cantidad -= 1
            for t in range(cantidad):
                nuevos_registros.append(row)

    df = df.append(nuevos_registros, ignore_index = True).drop(labels = ['Index','cantidad'], axis = 1)

    df = pd.concat((df, pd.get_dummies(df.dia, prefix = 'dia')), axis = 1)
    dias_presentes = df.dia.unique()
    if 1 not in dias_presentes:
        df = df.assign(dia_1 = 0)
    if 2 not in dias_presentes:
        df = df.assign(dia_2 = 0)
    if 3 not in dias_presentes:
        df = df.assign(dia_3 = 0)
    if 4 not in dias_presentes:
        df = df.assign(dia_4 = 0)
    if 5 not in dias_presentes:
        df = df.assign(dia_5 = 0)
    if 6 not in dias_presentes:
        df = df.assign(dia_6 = 0)
    if 7 not in dias_presentes:
        df = df.assign(dia_7 = 0)
    df = df.assign(dia_sin = lambda x: np.sin(2 * np.pi * x.dia / 7))
    df = df.assign(dia_cos = lambda x: np.cos(2 * np.pi * x.dia / 7))
    
    df = pd.concat((df, pd.get_dummies(df.mes, prefix = 'mes')), axis = 1)
    meses_presentes = df.mes.unique()
    if 1 not in meses_presentes:
        df = df.assign(mes_1 = 0)
    if 2 not in meses_presentes:
        df = df.assign(mes_2 = 0)
    if 3 not in meses_presentes:
        df = df.assign(mes_3 = 0)
    if 4 not in meses_presentes:
        df = df.assign(mes_4 = 0)
    if 5 not in meses_presentes:
        df = df.assign(mes_5 = 0)
    if 6 not in meses_presentes:
        df = df.assign(mes_6 = 0)
    if 7 not in meses_presentes:
        df = df.assign(mes_7 = 0)
    if 8 not in meses_presentes:
        df = df.assign(mes_8 = 0)
    if 9 not in meses_presentes:
        df = df.assign(mes_9 = 0)
    if 10 not in meses_presentes:
        df = df.assign(mes_10 = 0)
    if 11 not in meses_presentes:
        df = df.assign(mes_11 = 0)
    if 12 not in meses_presentes:
        df = df.assign(mes_12 = 0)
    df = df.assign(mes_sin = lambda x: np.sin(2 * np.pi * x.mes / 12))
    df = df.assign(mes_cos = lambda x: np.cos(2 * np.pi * x.mes / 12))

    return df[['cliente',
               'dia',
               'dia_1','dia_2','dia_3','dia_4','dia_5','dia_6','dia_7',
               'dia_sin','dia_cos',
               'mes',
               'mes_1','mes_2','mes_3','mes_4','mes_5','mes_6','mes_7','mes_8','mes_9','mes_10','mes_11','mes_12',
               'mes_sin','mes_cos',
               'prod']]

def retirar_data_validacion(df):
    # No hay manera de saber qué registros pertenecen a los días 10 al 14 de mayo, así que se va a considerar todo mayo como validación
    return df[~((df.anho == '2021') & (df.mes == 'Mayo'))]

def preparar_dataset(df):
    df = df.drop(labels = ['_id','fecNacimiento','sexo','monto','descuento'], axis = 1)
    df = retirar_data_validacion(df)
    df = corregir_dtypes_dataframe(df)
    df = limpiar_dataset(df)
    df = generar_variables(df)
    return df
 
def generar_lista_variantes():
    variantes = {1: ['dia','prod'],
                 2: ['dia_1','dia_2','dia_3','dia_4','dia_5','dia_6','dia_7','prod'],
                 3: ['dia_sin','dia_cos','prod'],
                 4: ['mes','prod'],
                 5: ['mes_1','mes_2','mes_3','mes_4','mes_5','mes_6','mes_7','mes_8','mes_9','mes_10','mes_11','mes_12','prod'],
                 6: ['mes_sin','mes_cos','prod'],
                 7: ['dia','mes','prod'],
                 8: ['dia','mes_1','mes_2','mes_3','mes_4','mes_5','mes_6','mes_7','mes_8','mes_9','mes_10','mes_11','mes_12','prod'],
                 9: ['dia','mes_sin','mes_cos','prod'],
                 10: ['dia_1','dia_2','dia_3','dia_4','dia_5','dia_6','dia_7','mes','prod'],
                 11: ['dia_1','dia_2','dia_3','dia_4','dia_5','dia_6','dia_7','mes_1','mes_2','mes_3','mes_4','mes_5','mes_6','mes_7','mes_8','mes_9','mes_10','mes_11','mes_12','prod'],
                 12: ['dia_1','dia_2','dia_3','dia_4','dia_5','dia_6','dia_7','mes_sin','mes_cos','prod'],
                 13: ['dia_sin','dia_cos','mes','prod'],
                 14: ['dia_sin','dia_cos','mes_1','mes_2','mes_3','mes_4','mes_5','mes_6','mes_7','mes_8','mes_9','mes_10','mes_11','mes_12','prod'],
                 15: ['dia_sin','dia_cos','mes_sin','mes_cos','prod']}
    return variantes

def oversampling(X, y):
    return SMOTE().fit_resample(X, y)
 
def aplicar_grid_search(X, y, modelo, params):
    if len(X) > 20:
        cv = StratifiedKFold(10)
    else:
        cv = LeaveOneOut()
    gs = GridSearchCV(estimator = modelo,
                      param_grid = params,
                      cv = cv,
                      scoring = ['accuracy','f1_weighted'],
                      refit = 'f1_weighted',
                      return_train_score = True,
                      verbose = 2,
                      n_jobs = -1)
    gs.fit(X, y)
    return (gs.best_estimator_,
            dict(test_f1_weighted = gs.best_score_,
                 train_f1_weighted = gs.cv_results_['mean_train_f1_weighted'][gs.best_index_],
                 test_accuracy = gs.cv_results_['mean_test_accuracy'][gs.best_index_],
                 train_accuracy = gs.cv_results_['mean_train_accuracy'][gs.best_index_]
                )
           )
 
def entrenar_LR(X, y):
    print('algoritmo LR')
    modelo = LogisticRegression(max_iter = 1000)
    params = dict(C = np.logspace(-6,2,9), solver = ('newton-cg','sag','saga','lbfgs'))
    clasificador = aplicar_grid_search(X, y, modelo, params)
    return clasificador

def entrenar_SVM(X, y):
    print('algoritmo SVM')
    modelo = SVC()
    params = dict(C = np.logspace(-7,2,9), gamma = np.logspace(-7,2,9))
    clasificador = aplicar_grid_search(X, y, modelo, params)
    return clasificador

def entrenar_RF(X, y):
    print('algoritmo RF')
    modelo = DecisionTreeClassifier(n_estimators = 70)
    params = dict(max_depth = range(3,11,2), min_samples_split = range(3,11,2))
    clasificador = aplicar_grid_search(X, y, modelo, params)
    return clasificador

def entrenar_NB(X, y):
    print('algoritmo NB')
    modelo = GaussianNB()
    params = dict(var_smoothing = np.logspace(-11,-7,5))
    clasificador = aplicar_grid_search(X, y, modelo, params)
    return clasificador

def entrenar_KNN(X, y):
    print('algoritmo KNN')
    modelo = KNeighborsClassifier()
    params = dict(n_neighbors = range(2, 10, 3), weights = ('uniform','distance'), p = (1,2))
    clasificador = aplicar_grid_search(X, y, modelo, params)
    return clasificador

def busqueda_mejor_variante_algoritmo_hiperparametros(producto, df_producto):
    df_producto.loc[df_producto.prod == producto, 'prod'] = 1
    df_producto.loc[df_producto.prod != producto, 'prod'] = 0
    X = df_producto.drop(labels = 'prod', axis = 1)
    y = df_producto['prod'].astype('int')
    X, y = oversampling(X, y)
    mejores_clasificadores_este_producto_y_cliente = list()
    for cod_variante, variante in variantes.items():
        print('cod_variante {}'.format(cod_variante))
        for entrenar_algoritmo in (entrenar_NB, entrenar_RF, entrenar_LR, entrenar_SVM, entrenar_KNN):
            mejor_clasificador, puntajes = entrenar_algoritmo(X, y)
            mejores_clasificadores_este_producto_y_cliente.append(dict(cod_variante = cod_variante,
                                                                       clasificador = mejor_clasificador,
                                                                       puntajes = puntajes)
                                                                 )
    mejor_de_mejores = sorted(mejores_clasificadores_este_producto_y_cliente, key = criterio_ordenacion, reverse = True)[0]
    return mejor_de_mejores

def entrenar(df, variantes):
    global clasificadores_por_cliente
    criterio_ordenacion = lambda x: (x['puntajes']['test_f1_weighted'], x['puntajes']['test_accuracy'], x['puntajes']['train_f1_weighted'], x['puntajes']['train_accuracy'])    
    lista_clientes = sorted(df.cliente.unique(), key = lambda cod_cli: int(cod_cli[3:]))
    lista_clientes.remove('SID1')
    clasificadores_por_cliente = dict()
    for cliente in lista_clientes:
        print('cliente {}'.format(cliente))
        df_cliente = df[df.cliente == cliente].copy()
        lista_productos_este_cliente = sorted(df_cliente.prod.unique(), key = lambda cod_prod: int(cod_prod[3:]))
        n_clases = len(lista_productos_este_cliente)
        clasificador_por_producto_este_cliente = dict()
        if n_clases == 1:
            continue
        elif n_clases == 2:
            producto_1, producto_2 = lista_productos_este_cliente
            mejor_de_mejores = busqueda_mejor_variante_algoritmo_hiperparametros(producto_2, df_cliente.copy())
            mejor_de_mejores['clases_prod'] = {0: producto_1, 1: producto_2}
            clasificadores_por_cliente[cliente] = dict(n_clases = n_clases, solucion = mejor_de_mejores)
        elif n_clases > 2:
            for producto in lista_productos_este_cliente:
                mejor_de_mejores = busqueda_mejor_variante_algoritmo_hiperparametros(producto, df_cliente.copy())
                clasificador_por_producto_este_cliente[producto] = mejor_de_mejores
            clasificadores_por_cliente[cliente] = dict(n_clases = n_clases, solucion = clasificador_por_producto_este_cliente)
    return clasificadores_por_cliente

def main():
    df = crear_dataframe_desde_mongodb()
    df = preparar_dataset(df)
    variantes = generar_lista_variantes()
    c = entrenar(df, variantes)
    return c

clasificadores_por_cliente = None
rs = 10
np.random.seed(rs)
ans = main()

In [ ]:
import pickle
pickle_out = open('trabajo_v2.pkl', 'wb')
pickle.dump(ans, pickle_out)
pickle_out.close()

In [ ]:
import pickle
pickle_in = open('trabajo_v2.pkl', 'rb')
ans = pickle.load(pickle_in)
pickle_in.close()

In [ ]:
from sklearn.model_selection import GridSearchCV
clientes_cod = {1:'SID2',2:'SID3',3:'SID5',4:'SID6',5:'SID7',6:'SID8',7:'SID9',8:'SID10',9:'SID12',10:'SID13',11:'SID14',12:'SID15',13:'SID17',14:'SID18',15:'SID20',16:'SID21',17:'SID22',18:'SID23',19:'SID24',20:'SID25',21:'SID26',22:'SID27',23:'SID28',24:'SID29',25:'SID30',26:'SID32',27:'SID33',28:'SID34',29:'SID35',30:'SID36',31:'SID38',32:'SID39',33:'SID40',34:'SID41',35:'SID42',36:'SID43',37:'SID44',38:'SID45',39:'SID46',40:'SID47',41:'SID48',42:'SID49',43:'SID50',44:'SID52',45:'SID53',46:'SID54',47:'SID55',48:'SID56',49:'SID57',50:'SID59',51:'SID61',52:'SID65',53:'SID66',54:'SID67',55:'SID68',56:'SID70',57:'SID71',58:'SID72',59:'SID73',60:'SID74',61:'SID75',62:'SID76',63:'SID77',64:'SID78',65:'SID79',66:'SID80',67:'SID81',68:'SID82',69:'SID83',70:'SID84',71:'SID85',72:'SID86',73:'SID87',74:'SID88',75:'SID89',76:'SID90',77:'SID91',78:'SID92',79:'SID93'}
n_features_variantes = {1: 1, 7: 2, 2: 3}
clasificadores = dict()
for cod_cliente, cliente_gs in enumerate(ans):
    print(clientes_cod[cod_cliente + 1])
    if type(cliente_gs[0]) == GridSearchCV:
        mejor_gs = sorted(cliente_gs, key = lambda gs: (gs.best_score_, gs.cv_results_['mean_test_accuracy'][gs.best_index_], gs.cv_results_['mean_train_f1_weighted'][gs.best_index_], gs.cv_results_['mean_train_accuracy'][gs.best_index_]), reverse = True)[0]
        mejor_clasificador = mejor_gs.best_estimator_
        variante = mejor_gs
    else:
        mejor_clasificador = cliente_gs[0]
        variante = None
    clasificadores[clientes_cod[cod_cliente + 1]] = (variante, mejor_clasificador)
    

SID2
SID3
SID5
SID6
SID7
SID8
SID9
SID10
SID12
SID13
SID14
SID15
SID17
SID18
SID20
SID21
SID22
SID23
SID24
SID25
SID26
SID27
SID28
SID29
SID30
SID32
SID33
SID34
SID35
SID36
SID38
SID39
SID40
SID41
SID42
SID43
SID44
SID45
SID46
SID47
SID48
SID49
SID50
SID52
SID53
SID54
SID55
SID56
SID57
SID59
SID61
SID65
SID66
SID67
SID68
SID70
SID71
SID72
SID73
SID74
SID75
SID76
SID77
SID78
SID79
SID80
SID81
SID82
SID83
SID84
SID85
SID86
SID87
SID88
SID89
SID90
SID91
SID92
SID93


In [ ]:
clasificadores

{'SID10': (2,
  OneVsRestClassifier(estimator=LogisticRegression(C=1e-06, solver='sag'))),
 'SID12': (1, OneVsOneClassifier(estimator=GaussianNB(var_smoothing=1e-11))),
 'SID13': (1,
  OneVsOneClassifier(estimator=DecisionTreeClassifier(max_depth=2))),
 'SID14': (2,
  OneVsOneClassifier(estimator=LogisticRegression(C=1e-06, solver='sag'))),
 'SID15': (1, OneVsOneClassifier(estimator=GaussianNB(var_smoothing=1e-11))),
 'SID17': (1,
  OneVsOneClassifier(estimator=DecisionTreeClassifier(max_depth=2))),
 'SID18': (1,
  OneVsOneClassifier(estimator=DecisionTreeClassifier(max_depth=2))),
 'SID2': (3,
  OneVsOneClassifier(estimator=LogisticRegression(C=100.0, solver='newton-cg'))),
 'SID20': (None, 205),
 'SID21': (1,
  OneVsOneClassifier(estimator=DecisionTreeClassifier(max_depth=2))),
 'SID22': (1, OneVsOneClassifier(estimator=GaussianNB(var_smoothing=1e-11))),
 'SID23': (1, OneVsOneClassifier(estimator=GaussianNB(var_smoothing=1e-11))),
 'SID24': (1, OneVsOneClassifier(estimator=GaussianNB

In [ ]:
pickle_out = open('clasificadores_v2.pkl', 'wb')
pickle.dump(clasificadores, pickle_out)
pickle_out.close()

In [ ]:
from sklearn.metrics import f1_score

df = crear_dataframe_desde_mongodb()
df = df.drop(labels = ['_id','fecNacimiento','sexo','monto','descuento'], axis = 1)

train = df[~((df.anho == '2021') & (df.mes == 'Mayo'))]
train = train.drop(labels = ['anho'], axis = 1)
train = corregir_dtypes_dataframe(train)
train = limpiar_dataset(train)
train = generar_variables(train)

test = df[((df.anho == '2021') & (df.mes == 'Mayo'))]
test = test.drop(labels = ['anho'], axis = 1)
test = corregir_dtypes_dataframe(test)
test = limpiar_dataset(test)
test = generar_variables(test)

clasificadores = obtener_clasificadores()
variantes = generar_lista_variantes()

curvas_por_cliente = list()
for cliente, (cod_variante, clasificador) in clasificadores.items():
    train_cliente = train[train.cliente == cliente][variantes[cod_variante]]
    X_train_cliente = train_cliente.drop(labels = 'prod', axis = 1)
    y_train_cliente = train_cliente['prod'].astype('int')
    
    test_cliente = test[test.cliente == cliente][variantes[cod_variante]]
    X_test_cliente = test_cliente.drop(labels = 'prod', axis = 1)
    y_test_cliente = test_cliente['prod'].astype('int')
    
    cantidad_training_samples = len(X_train_cliente)
    curva_aprendizaje = list()
    for train_samples in range(cantidad_training_samples):
        clasificador.fit(X_train_cliente.head(train_samples + 1), y_train_cliente.head(train_samples + 1))
        y_pred = clasificador.predict(X_test_cliente)
        score = f1_score(y_test_cliente, y_pred, average = 'weighted')
        curva_aprendizaje.append((train_samples, score))
    curvas_por_cliente.append(curvas_por_cliente)